In [11]:
import numpy as np 
import pandas as pd
import sklearn 
import tensorflow as tf 
import nltk
import gensim
import json

### Load Data

In [9]:
##TO-DO: Read dataset.json as a dictionary object 
with open('dataset.json', 'r') as file:
    json_str = file.read()
    json_dic = json.loads(json_str)

In [10]:
##TO-DO: Convert dictionary object to pandas dataframe 
df = pd.DataFrame.from_dict(json_dic).T
for i in range (len(df.index)):
    df['post_tokens'].iloc[i] = " ".join(df['post_tokens'].iloc[i])
df.head()

annotators  \
1179055004553900032_twitter  [{'label': 'normal', 'annotator_id': 1, 'targe...   
1179063826874032128_twitter  [{'label': 'normal', 'annotator_id': 1, 'targe...   
1178793830532956161_twitter  [{'label': 'normal', 'annotator_id': 4, 'targe...   
1179088797964763136_twitter  [{'label': 'hatespeech', 'annotator_id': 1, 't...   
1179085312976445440_twitter  [{'label': 'hatespeech', 'annotator_id': 4, 't...   

                                                 post_id  \
1179055004553900032_twitter  1179055004553900032_twitter   
1179063826874032128_twitter  1179063826874032128_twitter   
1178793830532956161_twitter  1178793830532956161_twitter   
1179088797964763136_twitter  1179088797964763136_twitter   
1179085312976445440_twitter  1179085312976445440_twitter   

                                                                   post_tokens  \
1179055004553900032_twitter  i dont think im getting my baby them white 9 h...   
1179063826874032128_twitter  we cannot continue calling ourselves feminists...   
1178793830532956161_twitter                      nawt yall niggers ignoring me   
1179088797964763136_twitter  <user> i am bit confused coz chinese ppl can n...   
1179085312976445440_twitter  this bitch in whataburger eating a burger with...   

                                                                    rationales  
1179055004553900032_twitter                                                 []  
1179063826874032128_twitter                                                 []  
1178793830532956161_twitter                                                 []  
1179088797964763136_twitter  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...  
1179085312976445440_twitter  [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...

In [18]:
from nltk.tokenize import word_tokenize

nltk.download('punkt')

df['tokenized'] = df.apply(lambda row: nltk.word_tokenize(row['post_tokens']), axis=1)

print(df['tokenized'])




[nltk_data] Downloading package punkt to /Users/23brianc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1179055004553900032_twitter    [i, dont, think, im, getting, my, baby, them, ...
1179063826874032128_twitter    [we, can, not, continue, calling, ourselves, f...
1178793830532956161_twitter                  [nawt, yall, niggers, ignoring, me]
1179088797964763136_twitter    [<, user, >, i, am, bit, confused, coz, chines...
1179085312976445440_twitter    [this, bitch, in, whataburger, eating, a, burg...
13851720_gab                   [laura, loomer, raped, me, while, screaming, a...
24198545_gab                   [and, this, is, why, i, end, up, with, nigger,...
14567516_gab                            [nogs, jews, and, dykes, how, enriching]
19679648_gab                   [guilty, until, proven, innocent, unless, you,...
15858421_gab                   [tired, i, can, not, support, abortion, from, ...
23844035_gab                   [<, number, >, <, number, >, <, percent, >, of...
1089312238102609921_twitter    [tariq, nasheed, beating, the, fuck, out, of, ...
1159278048069464065_twitter 

### Pre-Process Data

In [44]:
from itertools import chain
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

def clean_text(text_features):
    
    text_features = text_features.str.lower()

    stop_words = stopwords.words('english')
    
    # df['tweet_without_stopword'] = text_features.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    df['tweet_without_stopword'] = text_features.apply(lambda x:[word for word in x.split() if word not in (stop_words)])

    tweets_without_stopword = list(df['tweet_without_stopword']) 

    lemmatized = []
    
    for words in tweets_without_stopword:
        temp = []

        for w in words:
            temp.append(lemmatizer.lemmatize(w))
            
        lemmatized.append(temp)     
        

    df['text_lemmatized'] = lemmatized
    
    """
    pre-process text features of our dataset:  
    - Case Lowering
    - Tokenization
    - Stop Word Removal
    - Any other you want, including TF-IDF filtering 
 
    Paramaters
    ---------
    text_features: text (tweet) data in our dataframe.
    """

    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training

 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/23brianc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/23brianc/nltk_data...


In [45]:
clean_text(df['post_tokens'])

df.head()

annotators  \
1179055004553900032_twitter  [{'label': 'normal', 'annotator_id': 1, 'targe...   
1179063826874032128_twitter  [{'label': 'normal', 'annotator_id': 1, 'targe...   
1178793830532956161_twitter  [{'label': 'normal', 'annotator_id': 4, 'targe...   
1179088797964763136_twitter  [{'label': 'hatespeech', 'annotator_id': 1, 't...   
1179085312976445440_twitter  [{'label': 'hatespeech', 'annotator_id': 4, 't...   

                                                 post_id  \
1179055004553900032_twitter  1179055004553900032_twitter   
1179063826874032128_twitter  1179063826874032128_twitter   
1178793830532956161_twitter  1178793830532956161_twitter   
1179088797964763136_twitter  1179088797964763136_twitter   
1179085312976445440_twitter  1179085312976445440_twitter   

                                                                   post_tokens  \
1179055004553900032_twitter  i dont think im getting my baby them white 9 h...   
1179063826874032128_twitter  we cannot continue calling ourselves feminists...   
1178793830532956161_twitter                      nawt yall niggers ignoring me   
1179088797964763136_twitter  <user> i am bit confused coz chinese ppl can n...   
1179085312976445440_twitter  this bitch in whataburger eating a burger with...   

                                                                    rationales  \
1179055004553900032_twitter                                                 []   
1179063826874032128_twitter                                                 []   
1178793830532956161_twitter                                                 []   
1179088797964763136_twitter  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   
1179085312976445440_twitter  [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...   

                                                                     tokenized  \
1179055004553900032_twitter  [i, dont, think, im, getting, my, baby, them, ...   
1179063826874032128_twitter  [we, can, not, continue, calling, ourselves, f...   
1178793830532956161_twitter                [nawt, yall, niggers, ignoring, me]   
1179088797964763136_twitter  [<, user, >, i, am, bit, confused, coz, chines...   
1179085312976445440_twitter  [this, bitch, in, whataburger, eating, a, burg...   

                                                        tweet_without_stopword  \
1179055004553900032_twitter  dont think im getting baby white 9 two white j...   
1179063826874032128_twitter  cannot continue calling feminists rights womxn...   
1178793830532956161_twitter                         nawt yall niggers ignoring   
1179088797964763136_twitter  <user> bit confused coz chinese ppl access twi...   
1179085312976445440_twitter  bitch whataburger eating burger top bun holdin...   

                                                               text_lemmatized  
1179055004553900032_twitter  [d, o, n, t,  , t, h, i, n, k,  , i, m,  , g, ...  
1179063826874032128_twitter  [c, a, n, n, o, t,  , c, o, n, t, i, n, u, e, ...  
1178793830532956161_twitter  [n, a, w, t,  , y, a, l, l,  , n, i, g, g, e, ...  
1179088797964763136_twitter  [<, u, s, e, r, >,  , b, i, t,  , c, o, n, f, ...  
1179085312976445440_twitter  [b, i, t, c, h,  , w, h, a, t, a, b, u, r, g, ...

In [46]:
from nltk.stem import WordNetLemmatizer

def vectorize_text(cleaned_text, use_word2vec=True):
    """
    use Word2Vec or Sentence2Vec to vectorize the cleaned text data
 
    Paramaters
    ---------
    cleaned_text: (cleaned) text (tweet) data in our dataframe.
    """
    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training


### Model Training: KNN/Random Forest/RNNs

In [ ]:
from sklearn.model_selection import train_test_split

## TO-DO: split our data into train and test data

##### Using KNN
###### Refer to this resource: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [ ]:
## TO-DO: Determine initial value of parameters (especially K)
k = 0

## TO-DO: determine the best distance computation metric
candidate_metrics = ['euclidean','cosine','manhattan']
best_metric = ''




In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## TO-DO: fit a KNN Classifier with distance metric and K 

In [ ]:
# TO-DO: calculating accuracy
from sklearn.metrics import accuracy_score


In [ ]:
## TO-DO: optimize K with hyperparameter tuning

from sklearn.model_selection import cross_val_score


## TO-DO: Visualize results to determine best K


In [ ]:
## TO-DO: repeat training with optimized K

##### Using XGBoost/Random Forest
###### Refer to this resource: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
## TO-DO: Train model naively, with default parameters
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## TO-DO: Evaluate base model
from sklearn.metrics import accuracy_score



In [ ]:
##TO-DO: Hyperparameter Tuning via RandomizedSearch & GridSearch 

## get current parameters and their values 
from pprint import pprint


## Randomized Search 
from sklearn.model_selection import RandomizedSearchCV
random_grid = {}
rf_random = RandomizedSearchCV()

## Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {}
grid_search = GridSearchCV()

##### Using LSTMs
###### Refer to this resource: https://www.youtube.com/watch?v=8HyCNIVRbSU&ab_channel=TheA.I.Hacker-MichaelPhi

In [ ]:
from tensorflow.python.keras.layers import LSTM, Activation, Dropout, Dense, Embedding
from tensorflow.python.keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences


In [ ]:
## veryyyyyy basic architecture. TO-DO: complete and expand

model = Sequential()
model.add(Embedding())
model.add(LSTM())
model.add(Dense(2, activation='softmax'))

model.compile(loss='',optimizer='', metrics=['accuracy'])

In [ ]:
epochs = 0
batch_size = 0

## TO-DO: determine training parameters and complete call
model.fit()

In [ ]:
## TO-DO: evaluate function 
model.evaluate()

### Using BERT
###### Refer to: http://jalammar.github.io/illustrated-bert/ 
###### Refer to: https://medium.com/@parthdholakiya180/twitter-hate-detection-using-bert-e7682b2d0a0c 

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer